# CIFAR-100 Classification

In [ ]:
# Import neccessary packages here
import numpy as np
import keras
from keras import backend as K
from keras.datasets import cifar100
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.preprocessing import image

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

169009152/169001437 [==============================] - 2s 0us/step


In [ ]:
x_train.shape

(50000, 32, 32, 3)

In [ ]:
y_train.max()

99

In [ ]:
num_classes = 100
img_rows, img_cols = 32, 32

In [ ]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 3, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 3, img_rows, img_cols)
    input_shape = (3, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 3)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 3)
    input_shape = (img_rows, img_cols, 3)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [ ]:
y_test[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [ ]:
# Design the model architecture here
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', use_bias=True, 
                  bias_initializer='zeros', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu', use_bias=True, 
                  bias_initializer='zeros'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu', use_bias=True, 
                  bias_initializer='zeros'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))


model.add(Flatten())
model.add(Dense(128,kernel_regularizer=keras.regularizers.l2(0.001), activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64,kernel_regularizer=keras.regularizers.l2(0.001), activation='relu'))
model.add(Dense(32,kernel_regularizer=keras.regularizers.l2(0.001), activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_18 (Dropout)         (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 16, 16, 64)        18496     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_19 (Dropout)         (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 8, 8, 64)         

In [ ]:
# Compile the model architecture here
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [ ]:
batch_size = 100
epochs = 50

In [ ]:
datagen = image.ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True) 
 # prepare iterator
it_train = datagen.flow(x_train, y_train, batch_size=batch_size)

In [ ]:
callback = keras.callbacks.EarlyStopping(monitor='loss', patience=3)

In [ ]:
# Train the transfer learning model here
model.fit(it_train,
          epochs=epochs,
          verbose=1,
          validation_data = (x_test,y_test), callbacks=callback)

Epoch 1/50
500/500 [==============================] - 21s 41ms/step - loss: 4.7364 - accuracy: 0.0173 - val_loss: 4.2441 - val_accuracy: 0.0538
Epoch 2/50
500/500 [==============================] - 20s 40ms/step - loss: 4.1970 - accuracy: 0.0526 - val_loss: 4.0992 - val_accuracy: 0.0785
Epoch 3/50
500/500 [==============================] - 20s 41ms/step - loss: 4.0434 - accuracy: 0.0799 - val_loss: 3.9198 - val_accuracy: 0.1093
Epoch 4/50
500/500 [==============================] - 20s 40ms/step - loss: 3.9307 - accuracy: 0.0969 - val_loss: 3.8583 - val_accuracy: 0.1220
Epoch 5/50
500/500 [==============================] - 20s 41ms/step - loss: 3.8458 - accuracy: 0.1089 - val_loss: 3.8147 - val_accuracy: 0.1296
Epoch 6/50
500/500 [==============================] - 20s 41ms/step - loss: 3.7721 - accuracy: 0.1217 - val_loss: 3.7167 - val_accuracy: 0.1458
Epoch 7/50
500/500 [==============================] - 20s 40ms/step - loss: 3.7112 - accuracy: 0.1373 - val_loss: 3.6331 - val_accuracy:

### Testing

In [ ]:
# Test the transfer learning model with testing set
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

### Save